# Text mining & Search Project

### Università degli Studi di Milano-Bicocca  2020/2021

**Luzzi Federico** (matricola) **Peracchi Marco** 800578

# Text Processing & Representation

In questa fase del progetto vengono applicate le fasi del text processing, come la tokenization, e molte altre, al fine di permettere la fase successiva di text representation.

In [1]:
# Librerie base
import nltk
import pandas as pd
import re
import os
import string
import matplotlib.pyplot as plt
import sklearn
from wordcloud import WordCloud

In [2]:
# Librerie per la text tokenization
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  WordPunctTokenizer
from nltk.tokenize import  BlanklineTokenizer

In [3]:
# Librerie per stemming e lemmatization
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [4]:
# Librerie per text representation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument # doc2vec

In [5]:
# Download dei contenuti necessari
nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words("english")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Lettura del dataset
df = pd.read_csv("data/labeled_data.csv", sep = ',').drop("Unnamed: 0", axis=1)
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [7]:
# Testo esempio
text = df["tweet"][19999]
text

"RT @shakiraevanss: Criticize Amanda for saying the n word, sure, but don't make jokes about her sexual assault, don't be trash."

### Preprocessing

In [8]:
def preprocessing(text):
    text = text.lower() # Lowering case
    remove_url = re.sub(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', ' ', text) # Removing url
    remove_retweet = re.sub(r"@\w+", " ",remove_url) # Removing retweet
    remove_retweet = re.sub(r"&\w+", " ",remove_retweet) # Remove &amp
    remove_retweet = re.sub(r"\b([!#\$%&\\\(\)\*\+,-\./:;<=>\?@\[\]\^_`\{|\}\"~]+)\b", " ",remove_retweet) # Must check this one
    remove_retweet = re.sub(r"([a-z])\1{3,}", r"\1",remove_retweet)
    remove_punc = remove_retweet.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    final_text = re.sub(r'\d+', ' ', remove_punc) # Remove number 
    final_text = re.sub(r'\s+', ' ', final_text) # Removing exceeding spaces
    return final_text

In [113]:
text_prep = preprocessing(text)
text_prep

'rt criticize amanda for saying the n word sure but dont make jokes about her sexual assault dont be trash'

### Tokenization

In [9]:
def tokenization(text_clean, tok = "tweet"):
    if tok == "tweet": # TweetTokenizer
        tt = TweetTokenizer()
        tokenized_text = tt.tokenize(text_clean)
    elif tok == "wordpunct": # WordPunctTokenizer
        wpt = WordPunctTokenizer()
        tokenized_text = wpt.tokenize(text_clean)
    return tokenized_text

In [118]:
text_tok = tokenization(text_prep)
print(text_tok)

['rt', 'criticize', 'amanda', 'for', 'saying', 'the', 'n', 'word', 'sure', 'but', 'dont', 'make', 'jokes', 'about', 'her', 'sexual', 'assault', 'dont', 'be', 'trash']


### Removing stopwords

In [10]:
def remove_stopwords(tokenized_text):
    remove_sw = []
    for token in tokenized_text:
        stop_words.append("rt") # Added a stop words, RT of Retweet
        if token.lower() not in stop_words:
             remove_sw.append(token)
    return remove_sw

In [120]:
text_sw = remove_stopwords(text_tok)
print(text_sw)

['criticize', 'amanda', 'saying', 'n', 'word', 'sure', 'dont', 'make', 'jokes', 'sexual', 'assault', 'dont', 'trash']


### Stemming

In [11]:
def stemmer(tokenized_text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in tokenized_text]

In [123]:
print(stemmer(text_sw))

['critic', 'amanda', 'say', 'n', 'word', 'sure', 'dont', 'make', 'joke', 'sexual', 'assault', 'dont', 'trash']


### Lemmatization

In [12]:
# pos-tagging (1 document)
def pos_tagging(doc_token):
    return nltk.pos_tag(doc_token)

# convertion of pos tagging
def get_wordnet_pos(word_tag):
    if word_tag.startswith('J'):
        return "a"
    elif word_tag.startswith('V'):
        return "v"
    elif word_tag.startswith('R'):
        return "r"
    else:
        return "n"
    
# lemmatizer one word 
def lemmatizer(word):
    pos = get_wordnet_pos(word[1])
    wnl = WordNetLemmatizer()
    return wnl.lemmatize(word[0], pos = pos)

# lemmatizer one document
def lemmatizer_doc(doc_token):
    lemmas = [] 
    
    pos_document = pos_tagging(doc_token) # pos tagging
    for token in pos_document:
        lemmas.append( lemmatizer(token) ) # lemmatization x word
    
    return lemmas

In [125]:
print(lemmatizer_doc(text_sw))

['criticize', 'amanda', 'say', 'n', 'word', 'sure', 'dont', 'make', 'joke', 'sexual', 'assault', 'dont', 'trash']


### Salvataggio dataset

In [13]:
# Sum up function
def processing(text):
    text_prep = preprocessing(text)
    text_prep = tokenization(text_prep)
    text_prep = remove_stopwords(text_prep)
    text_prep = lemmatizer_doc(text_prep)
    #text_prep = stemmer(text_prep)
    text_prep = " ".join(text_prep)
    return text_prep

In [13]:
# Apply on all text
df_red = df
df_red["tweet_clean"] = df_red["tweet"].apply(lambda x : processing(x))

In [ ]:
# Salvataggio del dataset
df.to_csv("data/processing_dataset.csv")

In [17]:
# Lettura dataset
if 'processed_data.csv' not in os.listdir('data'):
    # load dataset
    df = pd.read_csv("data/labeled_data.csv", sep = ",").drop("Unnamed: 0", axis=1)
    # apply process_text
    df["tweet_clean"] = df["tweet"].apply(lambda x : processing(x))
    df.to_csv('data/processed_data.csv', index = False)
else:
    df = pd.read_csv("data/processed_data.csv", sep = ",")
print("Load & preprocessing --- DONE")

Load & preprocessing --- DONE


In [20]:
# drop tweet list na !!!
df["tweet_list"] = df["tweet_clean"].str.split(" ").tolist()
df.dropna(inplace = True)
df = df.reset_index(drop = True)# drop 2 NA because tweet is empty

### Bag of words

In [21]:
corpus = df["tweet_clean"]

In [25]:
# Using CountVectorizer
vectorizer = CountVectorizer(binary = True)
X = vectorizer.fit_transform(corpus)

# Shape
X.toarray().shape

(24781, 16946)

In [27]:
# Example of presence of a word
X.toarray()[1][X.toarray()[1] == 1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

###  Count Vector

In [28]:
# Using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

X.toarray()[1][X.toarray()[1] == 2]

array([2], dtype=int64)

In [33]:
X.toarray()[1][X.toarray()[1] == 2]

array([2], dtype=int64)

### Tf-idf

In [27]:
vectorizer = TfidfVectorizer(
    use_idf=True,
    max_features=100,
    min_df=5,
    max_df=0.501
    # , ngram_range=(1, 3) Il miglire rimane uni-gram, pessimo score del bi-gram
    )
tfidf = vectorizer.fit_transform(corpus).toarray()

In [33]:
tfidf[2]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.2325286 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.70019865,
       0.        , 0.        , 0.        , 0.44737228, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

### Doc2Vec
## CONTROLLARE PERCHE' NON SO COME SI FA

In [38]:
documents = [TaggedDocument (doc, [i]) for i, doc in enumerate(df["tweet_list"])]
    model = Doc2Vec (documents, vector_size=100, window=10, min_count=1)
    model.save("../models/d2v.model")

NameError: name 'Doc2Vec' is not defined